In [1]:
%load_ext autoreload
%autoreload 2

import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [2]:
import openai
import json
import time

from prompts import SIMPLE_TS_PROMPTS, SIMPLE_TS_WITH_REF_PROMPTS
from core.features.topic_segregation.utils import add_dicts, calculate_cost_gpt4_8k

from dotenv import load_dotenv
load_dotenv()

True

# need to do: sentence level text processing

In [3]:
def get_unique_from_raw_text(raw_text, sep = "\n"):

    raw_text = raw_text.replace("’", "'")
    raw_text_list = raw_text.split(sep)
    return list(set(raw_text_list))

def treat_output_for_json(input):
    out = "{" + input.split("{")[1].split("}")[0] + "}"
    # out = out.replace('\'', '"')
    # out = out.replace('\\"', '\'')
    return out

def chunk_raw_text_list(raw_text_list, max_len=3000):

    text_list = []
    char_len = 0
    ind_text_list = []

    for text in raw_text_list:

        l = len(text)
        char_len += l
        if char_len > max_len:
            text_list.append(ind_text_list)
            ind_text_list = []
            ind_text_list.append(text)
            char_len = 0
            char_len += l
        else:
            ind_text_list.append(text)

    text_list.append(ind_text_list)
    return text_list

### Topics Segregation without reference

In [4]:

def generate_topic_segregation(raw_text = None, n = 1, topics = None, complete_seg = True, model_id = "gpt-4-0613"):

    if raw_text is None:
        raise ValueError("raw_text must be provided")

    if topics is None:
        conversation = [{"role": "system", "content": SIMPLE_TS_PROMPTS}]

        user_prompt = f"""
        RAW_TEXT:

        //raw//

        {raw_text}

        //raw//

        OUTPUT:
        """

    elif topics is not None:
        conversation = [{"role": "system", "content": SIMPLE_TS_WITH_REF_PROMPTS}]

        n = len(topics)
        reference = {"number of classes to segregate": n, "topics": topics, "segregate all provided text": complete_seg}

        user_prompt = f"""
        RAW_TEXT:

        //raw//

        {raw_text}

        //raw//

        REFERENCE:
        {str(reference)}

        OUTPUT:
        """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}


In [16]:
raw_text = """
ODI Cricket World Cup Winners List From 1975 To 2023, Updated
Centre Approves 4% Hike In DA For Central Govt Employees
National Film Awards 2023 Winners List Updated
India Extends Curbs On Sugar Exports To Calm Local Prices
Increase In Minimum Support Prices (MSP) For Rabi Crops In 2024-25
Air India Express Reveals New Logo, Airplane Designs, To Add 50 Planes In 15 Months
India Likely To Sign Deal With US For 31 MQ-9B Drones By Feb 2024
India’s First Regional Rapid Transit System (RRTS) To Be Launched In UP
Global Remote Work Index, India Among World’s Worst Countries
Most Sixes In International Cricket 2023 (Updated)
Dr Meenesh Shah Elected To Board Of International Dairy Federation
List Of International Airports In India
Jericho Missile: A’Doomsday’ Weapon
Syed Mushtaq Ali Trophy 2023 Schedule
"""

result = generate_topic_segregation(raw_text=raw_text)

try:
    json_out = json.loads(result['output'])
except:
    out = "{" + result['output'].split("{")[1].split("}")[0] + "}"
    json_out = json.loads(out)

print(result['output'])
print(result['total_usage'])
print(json_out)

{
    "Cricket": ["ODI Cricket World Cup Winners List From 1975 To 2023, Updated", "Most Sixes In International Cricket 2023 (Updated)", "Syed Mushtaq Ali Trophy 2023 Schedule"],
    "Indian Government and Policies": ["Centre Approves 4% Hike In DA For Central Govt Employees", "India Extends Curbs On Sugar Exports To Calm Local Prices", "Increase In Minimum Support Prices (MSP) For Rabi Crops In 2024-25", "India Likely To Sign Deal With US For 31 MQ-9B Drones By Feb 2024"],
    "Awards": ["National Film Awards 2023 Winners List Updated"],
    "Aviation": ["Air India Express Reveals New Logo, Airplane Designs, To Add 50 Planes In 15 Months", "List Of International Airports In India"],
    "Transit Systems": ["India’s First Regional Rapid Transit System (RRTS) To Be Launched In UP"],
    "Work Culture": ["Global Remote Work Index, India Among World’s Worst Countries"],
    "Associations and Elections": ["Dr Meenesh Shah Elected To Board Of International Dairy Federation"],
    "Military"

### Topics Segregation with reference

In [20]:
raw_text = """
ODI Cricket World Cup Winners List From 1975 To 2023, Updated
Centre Approves 4% Hike In DA For Central Govt Employees
National Film Awards 2023 Winners List Updated
India Extends Curbs On Sugar Exports To Calm Local Prices
Increase In Minimum Support Prices (MSP) For Rabi Crops In 2024-25
Air India Express Reveals New Logo, Airplane Designs, To Add 50 Planes In 15 Months
India Likely To Sign Deal With US For 31 MQ-9B Drones By Feb 2024
India’s First Regional Rapid Transit System (RRTS) To Be Launched In UP
Global Remote Work Index, India Among World’s Worst Countries
Most Sixes In International Cricket 2023 (Updated)
Dr Meenesh Shah Elected To Board Of International Dairy Federation
List Of International Airports In India
Jericho Missile: A’Doomsday’ Weapon
Syed Mushtaq Ali Trophy 2023 Schedule
"""

topics = ["Sports", "Defence", "Aviation"]

result = generate_topic_segregation(raw_text=raw_text, topics=topics, complete_seg=False)

try:
    json_out = json.loads(result['output'])
except:
    out = "{" + result['output'].split("{")[1].split("}")[0] + "}"
    json_out = json.loads(out)

print(result['output'])
print(result['total_usage'])
print(calculate_cost_gpt4_8k(result['total_usage']))
print(json_out)

{
    "Sports": ["ODI Cricket World Cup Winners List From 1975 To 2023, Updated",
               "National Film Awards 2023 Winners List Updated",
               "Most Sixes In International Cricket 2023 (Updated)",
               "Syed Mushtaq Ali Trophy 2023 Schedule"],
    "Defence": ["India Likely To Sign Deal With US For 31 MQ-9B Drones By Feb 2024",
                "Jericho Missile: A’Doomsday’ Weapon"],
    "Aviation": ["Air India Express Reveals New Logo, Airplane Designs, To Add 50 Planes In 15 Months",
                 "List Of International Airports In India"]
}
{'prompt_tokens': 483, 'total_tokens': 630, 'completion_tokens': 147}
{'Sports': ['ODI Cricket World Cup Winners List From 1975 To 2023, Updated', 'National Film Awards 2023 Winners List Updated', 'Most Sixes In International Cricket 2023 (Updated)', 'Syed Mushtaq Ali Trophy 2023 Schedule'], 'Defence': ['India Likely To Sign Deal With US For 31 MQ-9B Drones By Feb 2024', 'Jericho Missile: A’Doomsday’ Weapon'], 'Aviat

In [21]:
print(calculate_cost_gpt4_8k(result['total_usage']))

0.02331


### large raw text topic segregation

In [5]:
_text = """
IND vs BAN live: India strike back after solid opening stand
Live: Israel deploys hundreds of tanks, soldiers on Gaza border
Live: Israel deploys hundreds of tanks, soldiers on Gaza border
Why sudden cardiac arrests are killing so many young Indians
Why sudden cardiac arrests are killing so many young Indians
Experts say a number of factors raging from unhealthy lifestyle to impact on the heart from Covid-19 could be behind the rising incidents of sudden cardiac death or SCD
Gaza hospital blast: Can we really know what happened?
Gaza hospital blast: Can we really know what happened?
After a devastating explosion at a Gaza hospital, Palestine claimed 471 died in the "Israeli airstrike". Despite Israeli denials, thousands protested across the Arab and Muslim world, condemning the incident. There are independent probing teams that could potentially resolve the matter by examining forensic evidence. However, creating such teams quickly and ensuring its acceptance by all sides involved, including Israel, the Biden administration, Hamas, presents significant challenges.
Satyendar Jain's interim bail extended
Satyendar Jain's interim bail extended
On this date, the court will consider his plea for regular bail in a money laundering case.
Injured Hardik Pandya leaves field during clash vs Bangladesh
Injured Hardik Pandya leaves field during clash vs Bangladesh
India's all-rounder Hardik Pandya was forced to leave the field due to an ankle injury during their match against Bangladesh in the ODI World Cup. The incident occurred during the ninth over, when Pandya attempted to halt a powerful drive from Bangladesh's opener, Litton Das. However, his right foot slipped, causing him to stumble and take a painful tumble to the ground. Pandya experienced discomfort and received strapping on his injured right ankle.
How a shiny China-made airport became a trap for Nepal
How a shiny China-made airport became a trap for Nepal
China called the project a signature of its cooperation with Nepal. Insiders and documents reveal the pitfalls of China’s infrastructure-at-any-cost model.
The most affordable Apple Pencil is here: 11 things to know
The most affordable Apple Pencil is here: 11 things to know
Photos: Families of Israeli hostages share their stories
Photos: Families of Israeli hostages share their stories
Israel-Hamas war: Families of Israeli hostages share their loved ones' lives
WhatsApp to allow users to use two accounts on one phone soon
WhatsApp to allow users to use two accounts on one phone soon
Photos: RapidX launch preparations in full swing
Photos: RapidX launch preparations in full swing
Ghaziabad gears up for the RapidX launch, emphasizing security and arrangements for the PM Narendra Modi's opening speech.
Netflix hikes prices, again, in these countries
Netflix hikes prices, again, in these countries
Israel-Hamas war: US carriers and troops send a signal of force
Israel-Hamas war: US carriers and troops send a signal of force
As the Israel-Hamas conflict continues to escalate, President Joe Biden has deployed military assets, including aircraft carriers and Marines, to the region. The US aims to warn Iran and its proxies against further involvement in the conflict. The aircraft carriers provide significant striking power with their fighter jets and missile systems, and could potentially assist Israel in defending itself against rocket attacks.
'Bumrah should have been given a break against Bangladesh'
'Bumrah should have been given a break against Bangladesh'
Jasprit Bumrah, who underwent a surgery in New Zealand for a back injury, made a comeback into the team almost two months back after a long injury layoff and needs to be handled carefully as he is a vital cog in Team India's World Cup campaign at home.
Will shift to my village to take up farming: Murmu
Will shift to my village to take up farming: Murmu
President Droupadi Murmu surprised everyone by revealing her plan to shift to her village and take up farming after her tenure ends. She expressed her desire to learn from farmers in Bihar about farming techniques and increasing crop productivity. She also mentioned her connection to Bihar through historical links between Odisha and Bihar. This was her first visit to Bihar, but she claimed to be familiar with the culture and lifestyle through her previous role as the governor of Jharkhand. Murmu is the first person from a tribal community to hold the office of President.
11,000-year-old statue of man clutching penis excavated
11,000-year-old statue of man clutching penis excavated
Archaeologists in Turkey have discovered ancient statues, dating back 11,000 years, that challenge our understanding of pre-agricultural communities. The statues include a life-sized sculpture of a man and a wild boar, shedding light on the ancient society's culture and practices. The sculptures were found at the Gobekli Tepe and Karahan Tepe temple complexes, which are among the oldest known in the world
Rajasthan MBBS student jumps off hostel bldg, dies
Rajasthan MBBS student jumps off hostel bldg, dies
A 21-year-old medical student in Rajasthan died by suicide by jumping from the sixth floor of her college hostel building. The student, Sudhanshi Singh, left a note on her palm before taking the extreme step. She lived alone in the hostel and was seen going towards the bathroom before jumping off the balcony. Her body was taken to the hospital where she was declared brought dead.
How losing JUST one hour of sleep can harm your kids
How losing JUST one hour of sleep can harm your kids
Losing just one hour of sleep can be detrimental to your children's well-being. A recent study in JAMA Network Open found that even in healthy children, a single hour of sleep deprivation had a significant impact. Kids who compromised their sleep quality for a week experienced reduced physical and mental health, struggled at school, and had a diminished quality of life. These findings emphasize the importance of maintaining consistent bedtimes for children to support their overall health and well-being.
India vs Bangladesh: Virat Kohli bowls in an ODI afer 6 years
India vs Bangladesh: Virat Kohli bowls in an ODI afer 6 years
Hardik Pandya pulled up while bowling the 9th over of Bangladesh innings with pain in his left leg, which was then taped by the physio. But the all-rounder couldn't continue and Kohli was called upon to bowl the last three deliveries of the over. The last time Kohli came on to bowl in an ODI was against Sri Lanka in Colombo back in August 2017.
"""

In [6]:
# convert raw_text to list and get unique values

unique_text_list = get_unique_from_raw_text(_text, sep="\n")
# print(unique_text_list)
print(len(unique_text_list))

34


In [ ]:
topics = ["Sports", "Defence", "Regional:India"]

raw_text = "\n".join(unique_text_list)
result = generate_topic_segregation(raw_text=raw_text, topics=topics, complete_seg=False)

try:
    json_out = json.loads(result['output'])
except:
    out = "{" + result['output'].split("{")[1].split("}")[0] + "}"
    json_out = json.loads(out)

# print(result['output'])
print(result['total_usage'])
print(calculate_cost_gpt4_8k(result['total_usage']))
print(json_out)

### processing text by chunking

In [121]:

# chunking the list into sublists with max length of characters in a chunk is 3000
chunked_text_list = chunk_raw_text_list(unique_text_list, max_len=3000)

# print(chunked_text_list)

print(len(chunked_text_list))
for i in chunked_text_list:
    print(len(i))

2
20
14


In [123]:
topics = ["Sports", "Defence", "Regional:India"]
final_dict = {topic:[] for topic in topics}
total_tokens = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}

for raw_text in chunked_text_list:
    raw_text = "\n".join(raw_text)

    result = generate_topic_segregation(raw_text=raw_text, topics=topics, complete_seg=False)
    output = result['output']
    tokens = result['total_usage']

    try:
        json_out = json.loads(output)
    except:
        print("GOT ERROR IN JSON LOADING")
        out = treat_output_for_json(output)
        json_out = json.loads(out)
    # print(json_out)
    final_dict = add_dicts(final_dict, json_out)
    total_tokens = add_dicts(total_tokens, tokens)

print(total_tokens)
print(calculate_cost_gpt4_8k(total_tokens))
for topic in topics:
    print(f"Topic: {topic}")
    print(final_dict[topic])
    print()

{'prompt_tokens': 1653, 'total_tokens': 2473, 'completion_tokens': 820}
Topic: Sports
["Hardik Pandya pulled up while bowling the 9th over of Bangladesh innings with pain in his left leg, which was then taped by the physio. But the all-rounder couldn't continue and Kohli was called upon to bowl the last three deliveries of the over. The last time Kohli came on to bowl in an ODI was against Sri Lanka in Colombo back in August 2017.", 'India vs Bangladesh: Virat Kohli bowls in an ODI afer 6 years', "Jasprit Bumrah, who underwent a surgery in New Zealand for a back injury, made a comeback into the team almost two months back after a long injury layoff and needs to be handled carefully as he is a vital cog in Team India's World Cup campaign at home.", "India's all-rounder Hardik Pandya was forced to leave the field due to an ankle injury during their match against Bangladesh in the ODI World Cup. The incident occurred during the ninth over, when Pandya attempted to halt a powerful drive fr